# Data loading 
Next cells are used to load the data we need.

In [1]:
import pandas as pd
import numpy as np 
import scipy.sparse as sps
import os
import sys
import matplotlib.pyplot as plt
sys.path.append('../../')


In [2]:
#data which contains users (row), items(col) and implicit interaction (data)
dataset = pd.read_csv('../data_train.csv')
dataset

,row,col,data
0,0,10080,1.0
1,0,19467,1.0
2,1,2665,1.0
3,1,7494,1.0
4,1,17068,1.0
...,...,...,...
113263,7945,2476,1.0
113264,7945,12319,1.0
113265,7945,21384,1.0
113266,7946,8699,1.0


# Data pre-processing
Pre-processing of data to check for missing users, items ecc...

In [3]:
users = dataset.row
items = dataset.col
data = dataset.data
URM_all = sps.coo_matrix((data, (users, items)))
URM_all = URM_all.tocsr() #fast row access -> fast access to users 
URM_all.shape

(7947, 25975)

In [4]:
ICM_df = pd.read_csv('../data_ICM_title_abstract.csv')
ICM_df

,row,col,data
0,0,1185,1.015524
1,0,2507,0.459024
2,0,3534,0.227742
3,0,8766,0.501549
4,0,10862,0.297011
...,...,...,...
490686,25974,12554,0.963016
490687,25974,13003,0.104613
490688,25974,16236,0.118760
490689,25974,18797,0.363301


In [5]:
items = ICM_df.row
features = ICM_df.col
data = ICM_df.data
ICM_all = sps.coo_matrix((data, (items, features)))
ICM_all = ICM_all.tocsr() #fast row access -> fast access to users 
ICM_all.shape

(25975, 20000)

In [6]:
from Base.Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.85)
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

In [7]:
URM_train_ICM_all = sps.vstack([URM_train*(1-0.289), ICM_all.T*0.289])

In [12]:
from GraphBased.RP3betaRecommender import RP3betaRecommender

recommender_rp3Beta = RP3betaRecommender(URM_train_ICM_all)
recommender_rp3Beta.fit(topK=430, alpha = 0.47, beta=0.27, implicit = False)
recommender_rp3Beta.W_sparse.data = np.power(recommender_rp3Beta.W_sparse.data, 0.89)

recommender_rp3Beta.URM_train = URM_train
evaluator_validation.evaluateRecommender(recommender_rp3Beta)

RP3betaRecommender: URM Detected 41 (0.15 %) cold users.
EvaluatorHoldout: Processed 4977 ( 100.00% ) in 4.18 sec. Users per second: 1191


({10: {'ROC_AUC': 0.18778292845502048,
   'PRECISION': 0.039923648784407365,
   'PRECISION_RECALL_MIN_DEN': 0.16127990342878892,
   'RECALL': 0.1578872381943214,
   'MAP': 0.07320597120146453,
   'MRR': 0.13735724012999506,
   'NDCG': 0.10852747599607207,
   'F1': 0.0637319284240203,
   'HIT_RATE': 0.3992364878440828,
   'ARHR': 0.15782647798922694,
   'NOVELTY': 0.005027894563417119,
   'AVERAGE_POPULARITY': 0.06864717092565009,
   'DIVERSITY_MEAN_INTER_LIST': 0.9941266239492663,
   'DIVERSITY_HERFINDAHL': 0.9993926879801396,
   'COVERAGE_ITEM': 0.538902791145332,
   'COVERAGE_ITEM_CORRECT': 0.05151106833493744,
   'COVERAGE_USER': 0.6262740656851642,
   'COVERAGE_USER_CORRECT': 0.18749213539700516,
   'DIVERSITY_GINI': 0.2347841803122822,
   'SHANNON_ENTROPY': 12.535652502933754}},
 'CUTOFF: 10 - ROC_AUC: 0.1877829, PRECISION: 0.0399236, PRECISION_RECALL_MIN_DEN: 0.1612799, RECALL: 0.1578872, MAP: 0.0732060, MRR: 0.1373572, NDCG: 0.1085275, F1: 0.0637319, HIT_RATE: 0.3992365, ARHR: 0

In [ ]:
test_users = pd.read_csv('data_target_users_test.csv')
test_users

In [ ]:
recommender = ItemKNNCBFRecommender(URM_all, ICM_all)
recommender.fit(topK=600, shrink=200, feature_weighting = 'TF-IDF')

user_id = test_users['user_id']
recommendations = recommender.recommend(user_id,cutoff = 10)


In [ ]:
for index in range(len(recommendations)):
    recommendations[index]=np.array(recommendations[index])
    
print(len(recommendations))

test_users['item_list']= recommendations
test_users['item_list'] = pd.DataFrame([str(line).strip('[').strip(']').replace("'","") for line in test_users['item_list']])

test_users
test_users.to_csv('submission.csv', index=False)
